# ETL Auxiliar Notebook
Illustrates the process of:
1. Extracting tweets through Tweepy into json.
2. Transformed it to a dataframe object using Pandas. 
3. Load the result to Big Query.

**Step 0:** Install required packages

In [1]:
!pip install tweepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 KB 3.9 MB/s eta 0:00:00


In [9]:
!pip install pandas-gbq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 49.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 7.0.0
    Uninstalling pyarrow-7.0.0:
      Successfully uninstalled pyarrow-7.0.0


**Step 1:** Import libraries

In [11]:
import tweepy
import pandas as pd
import yaml
from google.cloud import bigquery

**Step 2:** Read config yaml and setup credentials

In [12]:
with open("src/config.yaml", "r") as config:
        config = yaml.safe_load(config)

In [17]:
# Credentials
consumer_key = config['consumer_key']
consumer_secret =  config['consumer_secret']
access_token =  config['access_token']
access_token_secret =  config['access_token_secret']

In [3]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

**Step 3:** Extract and Transform

In [18]:
user = []
text = []
followers = []
location = []
verified = []
df = pd.DataFrame(columns=['user','text','followers','location','verified'])


# Extract
themes = ['travel','booking','hotel','trivago','airbnb','travel agency','travelling','vacation','instatravel','tourism','traveller','trip','journey','tour','tourist']

for theme in themes:
	# The search term
	query = '{} -filter:retweets'.format(theme)
	# Language code (follows ISO 639-1 standards)
	language = "en"
	# Calling the api
	results = api.search_tweets(q=query, lang=language, count=100)
	# Features
	for tweet in results:
		user.append(tweet.user.screen_name)
		text.append(tweet.text)
		followers.append(tweet.user.followers_count)
		location.append(tweet.user.location)
		verified.append(tweet.user.verified)
df['user'] = user
df['text'] = text
df['followers'] = followers
df['location'] = location
df['verified'] = verified

In [19]:
df

,user,text,followers,location,verified
0,socalmommablog,Free Subscription to DuJour Magazine!.\n Sign ...,8069,California,False
1,Cali_Kaylee,why did I decide to do travel work when I like...,630,,False
2,joyfulgirl1971,@kentuckygrandma @cranberryhorn @Booker4KY @La...,12,"Los Angeles, CA",False
3,ArchibaldPug,@nicktolhurst From where they are expected to ...,1425,"Rye, England",False
4,Getstuffed01010,@Jess06091913 @KeithWarren55 @SAHealth Do you ...,2,,False
...,...,...,...,...,...
1495,bigpurk14,@FootyScran @SpursOfficial Who the fuck is wat...,562,,False
1496,johnjones5089,@GarNob Who in their rite mind (unless you are...,24,"Dublin, Ireland",False
1497,RuhulAm91543090,@NagaPotineni @VFSGlobal @vfsglobalcare @India...,3,,False
1498,girl75219,@Kyiv And Americans will come and visit and sp...,1523,"Dallas, TX",False


**Step 4:** Load

In [20]:
#Imports google cloud client library and initiates BQ service
bigquery_client = bigquery.Client(project = config['project'])

In [10]:
df[['user','text']].to_gbq(destination_table = 'tweets.text',project_id = config['project'],if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 10106.76it/s]


In [ ]:
df.drop(columns='text').to_gbq(destination_table = 'tweets.users',project_id = config['project'],if_exists='replace)